### Read  English Text & Number from tables in PDF files 

#### Two main methods were used to extract text: 
1- Extract text from pdf files then used RegExr to extract wanted variables.   
2- Extract tables from pdf files and used slice to extract wanted variables. 


In [4]:
import pandas as pd
from tabula import read_pdf
from tabulate import tabulate
import PyPDF2
import os
import re
import time

#### The below functions extract text then put them in dataframes 

In [7]:

# Specify the PDF file path
columns= ['Cons2022','Stand2022','Cons2021','Stand2021']
profit_data= pd.DataFrame(columns=columns)
profit_data
folder_path='oman 2022\IncomeInstatment'
file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]

for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)
#     print(f"Opening PDF: {file_name}")

    # Open the PDF file
    pdf_file = open(pdf_path, 'rb')

# tabula.convert_into("AACT_2022_Yearly (Audited)_e/2_AACT_BalanceSheet_26022023_33.pdf", "output.csv", output_format="csv", pages='all')
# Use tabula to extract tables from the PDF

    tables = read_pdf(pdf_file,multiple_tables=True,columns=[100,50, 100, 300],pages='all')
    # print(tabulate(tables)
    # Print the extracted tables
#     for table_num, table_df in enumerate(tables, start=1):
    extract_profit(tables)
#         print(f"Table {table_num}:\n{table_df}")
     # Close the PDF file
    pdf_file.close()

3_AACT_IncomeStatement_26022023_33.pdf
3_AAIC_IncomeStatement_04062023_39_U.pdf
3_ABOB_IncomeStatement_09032023_41.pdf
3_ABRJ_IncomeStatement_17032023_4_U.pdf
3_AFAI_IncomeStatement_13092022_30.pdf
3_AFIC_IncomeStatement_13032023_32.pdf
3_AJSS_IncomeStatement_28022023_2.pdf


KeyboardInterrupt: 

In [4]:
def extract_profit(tables):
    global profit_data
    global file_name
    # Read first table to extract the wanted row
    table=tables[0]
    # Drop null rows from first columns
    table.dropna(subset=['Unnamed: 0'],inplace=True)
    # Take a loop to look for specific values by order 
    keywords=['Net Profit / (Loss) for the period', 'Net profit (loss)', 'Profit (loss) for period', 'Profit (loss)','Profit for the period']
#     columns= ['Cons2022','Stand2022','Cons2021','Stand2021']
#     profit_data= pd.DataFrame(columns=columns)

    for keyword in keywords:
#     matched_rows=test[test['Unnamed: 0'].str.contains(keyword, case=False)]
        matched_row=table[table['Unnamed: 0']==keyword]
        if not matched_row.empty:
            matched_row=matched_row.iloc[[0]]
            break
#         else:
#             print('Not Matched')


    print(file_name)
#     cols=matched_row.columns[:1]
    last_columns=matched_row.iloc[:,1:]
#     matched_row['combine']=last_columns.apply(' '.join, axis=1)
    matched_row['combine']=last_columns.apply(lambda row:' '.join(filter(lambda x:pd.notna(x),row)),axis=1)

    matched_row.drop(matched_row.iloc[:,1:-1], axis=1, inplace=True)
    matched_row=matched_row['combine'].str.split(expand=True)
    if len(matched_row.columns)>2:
        matched_row.rename(columns={0:'Cons2022',1:'Stand2022',2:'Cons2021',3:'Stand2021'},inplace=True)
        
    else:
        matched_row.rename(columns={0:'Cons2022', 1:'Cons2021'},inplace=True)
        

    matched_row.index=[file_name.split('_')[1]]
    profit_data=pd.concat([profit_data,matched_row],ignore_index=False).fillna(0)
#     profit_data=profit_data.append(matched_row).fillna(0)

In [1753]:
import re
import pandas as pd
# Define the keyword set you want to search for as create the dataframe. 
keywords = ['Total assets', 'Total current assets', 'Total liabilities', 'Total current liabilities']
columns= pd.MultiIndex.from_product([['2022','2021'], keywords])
data= pd.DataFrame(columns=columns)
    
def extract_info(text,file_name):
    # Input text containing the keyword
    input_text = str(text).split('\\n')
    global keywords
    global data

    # keyword='Total liabilities'
    for keyword in keywords:
    # Use regex to find the keyword and capture text after it
    # pattern = re.compile(f"{re.escape(keyword)}.*?\\b([\d,]+)\\b", re.IGNORECASE)
        pattern = re.compile(f"{re.escape(keyword)}.*?\\b([\d,]+).*?([\d,]+)\\b", re.IGNORECASE)

        match = pattern.search(str(input_text))

        if match:
            com_name=file_name.split('_')[1]
            data.at[com_name,('2022',keyword)]=match.group(1)
            data.at[com_name,('2021',keyword)]=match.group(2)
            
    #         captured_number1 = match.group(1)
    #         print(f"Captured number: {captured_number1}")
    #         captured_number2 = match.group(2)
    #         print(f"Capture number: {captured_number2}")
        else:
            print("Keyword not found.")


In [1754]:

import pandas as pd

# Specify the folder path
folder_path = 'oman 2022\Balancesheet'

# List of files in the folder
# file_list = os.listdir(folder_path)
file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]

for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)
    print(f"Opening PDF: {file_name}")

    # Open the PDF file
    pdf_file = open(pdf_path, 'rb')

    # Create a PDF reader object
    reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF file
    num_pages = len(reader.pages)

    # Iterate over the pages in the PDF file
    text=[]
    for page_num in range(num_pages):

        # Get the current page
        page = reader.pages[page_num]

        # Get the tables on the current page
        tables = page.extract_text().split('\n\n')

        # Print the tables
        for table in tables:
            text.append(table)
            extract_info(text,file_name)

    # Close the PDF file
    pdf_file.close()
    

Opening PDF: 2_AACT_BalanceSheet_26022023_33.pdf
Opening PDF: 2_AAIC_BalanceSheet_04062023_39_U.pdf
Keyword not found.
Keyword not found.
Opening PDF: 2_ABOB_BalanceSheet_09032023_41.pdf
Keyword not found.
Keyword not found.
Keyword not found.
Keyword not found.
Opening PDF: 2_ABRJ_BalanceSheet_17032023_4_U.pdf
Opening PDF: 2_AFAI_BalanceSheet_13092022_30.pdf
Opening PDF: 2_AFIC_BalanceSheet_13032023_32.pdf
Keyword not found.
Keyword not found.
Opening PDF: 2_AJSS_BalanceSheet_28022023_2.pdf
Opening PDF: 2_AMAN_BalanceSheet_03042023_6.pdf
Keyword not found.
Keyword not found.
Opening PDF: 2_AMAT_BalanceSheet_26022023_31.pdf
Keyword not found.
Keyword not found.
Keyword not found.
Keyword not found.
Opening PDF: 2_AMCI_BalanceSheet_13032023_1.pdf
Opening PDF: 2_AMII_BalanceSheet_28052023_41_U.pdf
Keyword not found.
Keyword not found.
Keyword not found.
Keyword not found.
Opening PDF: 2_AOFS_BalanceSheet_15032023_39.pdf
Keyword not found.
Keyword not found.
Opening PDF: 2_ATMI_BalanceShe

In [405]:
data.isnull().sum()

2022  Total assets                  0
      Total current assets         32
      Total liabilities             0
      Total current liabilities    32
2021  Total assets                  0
      Total current assets         32
      Total liabilities             0
      Total current liabilities    32
dtype: int64

In [1821]:
full_data= pd.concat([data,profit_data],axis=1)

In [1825]:
full_data.columns

Index([             ('2022', 'Total assets'),
            ('2022', 'Total current assets'),
               ('2022', 'Total liabilities'),
       ('2022', 'Total current liabilities'),
                    ('2021', 'Total assets'),
            ('2021', 'Total current assets'),
               ('2021', 'Total liabilities'),
       ('2021', 'Total current liabilities'),
                                  'Cons2022',
                                 'Stand2022',
                                  'Cons2021',
                                 'Stand2021'],
      dtype='object')

In [1833]:
full_data.rename(columns={"('2022', 'Total assets')":'Total assets 2022',"('2021', 'Total assets')":'Total assets 2021',\
                          "('2022', 'Total current assets')":'Total current assets 2022',"('2021', 'Total current assets')":'Total current assets 2021',\
                         "('2022', 'Total liabilities')":'Total liabilities 2022',"('2021', 'Total liabilities')":'Total liabilities 2021',\
                        "('2022', 'Total current liabilities')":'Total current liabilities 2022',"('2021', 'Total current liabilities')":'Total current liabilities 2021'
                          
                         ,},inplace=True)

In [1824]:
full_data.to_excel('Oman.xlsx')

In [1834]:
full_data.head()

,"(2022, Total assets)","(2022, Total current assets)","(2022, Total liabilities)","(2022, Total current liabilities)","(2021, Total assets)","(2021, Total current assets)","(2021, Total liabilities)","(2021, Total current liabilities)",Cons2022,Stand2022,Cons2021,Stand2021
AACT,"47,733,926","21,134,057","15,793,514","7,452,305","46,204,285","25,348,510","11,468,890","6,351,016","2,925,827",0,"6,272,905",0
AAIC,"48,831",NaN,"17,525",NaN,"46,049",NaN,"15,912",NaN,354,0,(842),0
ABOB,"3,075,466",NaN,"2,614,527",NaN,"3,052,556",NaN,"2,625,559",NaN,"33,090",0,"27,606",0
ABRJ,"281,775","78,183","137,154","48,326","289,777","77,831","156,151","57,112","16,519",0,"18,413",0
AFAI,"53,508,895","13,784,439","33,509,835","21,583,666","46,363,741","4,000,703","20,302,720","12,192,438","(5,907,917)","(168,736)","205,935","452,174"


In [1801]:
# To know how many files in folders
folder_path='oman 2022\Balancesheet'
balancesheet=[file.split('_')[1] for file in os.listdir(folder_path) if file.endswith('.pdf')]
len(balancesheet)

103

In [25]:
# To know how many files in folders
folder_path='oman 2022\IncomeInstatment'
incomes=[file.split('_')[1] for file in os.listdir(folder_path) if file.endswith('.pdf')]
len(incomes)+9

103

In [7]:
# Check the pdf file 

def check_files(tables,file_name):
    global correct_file
    global wrong_file
    if len(tables)>0:
        correct_file.append(file_name)
        result= True
    else: # file return empty tables
        wrong_file.append(file_name)
        result=False
    
    return result   

In [95]:
start_time=time.time()
# Specify the PDF file path
correct_file=[]
wrong_file=[]
keywords = ['Total_assets', 'Total_current_assets', 'Total_liabilities', 'Total_current_liabilities']
columns= pd.MultiIndex.from_product([['2020','2021'], keywords])
balance_data= pd.DataFrame(columns=columns)
folder_path='oman 2022\BalanceSheet'
# file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
file_list=['2_AMAN_BalanceSheet_03042023_6.pdf']

for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)
#     print(f"Opening PDF: {file_name}")

    # Open the PDF file
#     pdf_file = open(pdf_path, 'rb')

# Use tabula to extract tables from the PDF

    tables = read_pdf(pdf_path,multiple_tables=True,columns=[200,300,300,300],pages='all')
    
#     Print the extracted tables
#     for table_num, table_df in enumerate(tables, start=1):
#         extract_profit(tables)
#         print(f"Table {table_nu qm}:\n{table_df}"
    result=check_files(tables,file_name)
    if result:
#         asset,liability, current=
        extract_balance(tables)
      
        break
    else:
        continue
     # Close the PDF file
print('Total time:',round((time.time()-start_time)/60), 'in Minute') 
print(x)
#     pdf_file.close()

filling data went wrong: 2_AMAN_BalanceSheet_03042023_6.pdf
Total time: 0 in Minute
0


In [89]:
def extract_balance(tables):
    global file_name
    global balance_data
    # Read first table to extract the wanted row
    for i in range(len(tables)):
        
        table=tables[i]
        # Drop null rows from last columns(as reverse in Arabic tables)
        first_col=table.columns[0]
    #     table[last_col].dropna(inplace=True)
        table.dropna(subset=[first_col],inplace=True)

    #     table.dropna(subset=['Unnamed: 0'],inplace=True)
    #     matched_row=matched_row.iloc[[0]]
    #     matched_row=table.iloc[:,:-1]

    #     matched_row=matched_row.dropna(axis=1) # delete Null columns
        com_name=file_name.split('_')[1]
    #     columns= ['Cons2022','Stand2022','Cons2021','Stand2021']
    #     profit_data= pd.DataFrame(columns=columns)

    #     for keyword in keywords:
    #     matched_rows=test[test['Unnamed: 0'].str.contains(keyword, case=False)]
        Total_assets=table.loc[table[first_col]=='Total assets']
        Total_current_assets=table.loc[table[first_col]=='Total current assets']
        Total_liabilities=table.loc[table[first_col]=='Total liabilities'] 
        Total_current_liabilities=table.loc[table[first_col]=='Total current liabilities']
        borrow_current=table.loc[table[first_col]=='Borrowings, current']
        borrow_non_current=table.loc[table[first_col]=='Borrowings, non current']
    #     return Total_assets,Total_liabilities, Total_current_liabilities


        if table.shape[1]==3:
            if not Total_assets.empty:
                balance_data.at[com_name,('2020','Total_assets')]=Total_assets.iloc[0,1]
                balance_data.at[com_name,('2021','Total_assets')]=Total_assets.iloc[0,2]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_liabilities')]):
                balance_data.at[com_name,('2020','Total_liabilities')]=Total_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_liabilities')]=Total_liabilities.iloc[0,2]
            else:
                continue
            if not Total_current_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_current_liabilities')]):
                balance_data.at[com_name,('2020','Total_current_liabilities')]=Total_current_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_current_liabilities')]=Total_current_liabilities.iloc[0,2]
            else:
                continue

        elif table.shape[1]==4:
            if not Total_assets.empty:
                balance_data.at[com_name,('2020','Total_assets')]=Total_assets.iloc[0,1]
                balance_data.at[com_name,('2021','Total_assets')]=Total_assets.iloc[0,3]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_liabilities')]):
                balance_data.at[com_name,('2020','Total_liabilities')]=Total_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_liabilities')]=Total_liabilities.iloc[0,3]
            else:
                continue
            if not Total_current_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_current_liabilities')]):
                balance_data.at[com_name,('2020','Total_current_liabilities')]=Total_current_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_current_liabilities')]=Total_current_liabilities.iloc[0,3]
            else:
                continue

        elif table.shape[1]==5:
            if not Total_assets.empty:
                balance_data.at[com_name,('2020','Total_assets')]=Total_assets.iloc[0,1]
                balance_data.at[com_name,('2021','Total_assets')]=Total_assets.iloc[0,3]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_liabilities')]):
                balance_data.at[com_name,('2020','Total_liabilities')]=Total_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_liabilities')]=Total_liabilities.iloc[0,3]
            else:
                continue
            if not Total_current_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_current_liabilities')]):
                balance_data.at[com_name,('2020','Total_current_liabilities')]=Total_current_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_current_liabilities')]=Total_current_liabilities.iloc[0,3]
            else:
                continue

        else:
            print('filling data went wrong:',file_name)




In [96]:
tables[0]

,Unnamed: 0,Actuals/Omani Rial/Audited
0,"Statement of Financial position, Order of liqu...",31/12/2022 31/12/2021
1,STATEMENT OF FINANCIAL POSITION,NaN
2,ASSETS,NaN
3,Cash and bank balances,16 923
4,Receivables,"508,987 405,813"
5,Investment properties,"20,498,000 20,800,000"
6,Total assets,"21,007,003 21,206,736"
7,LIABILITIES,NaN
8,Accruals and other pay ables,"81,368 24,773"
9,Total liabilities,"81,368 24,773"


In [57]:
import re
import pandas as pd
# Define the keyword set you want to search for as create the dataframe. 
keywords = ['Borrowings, current', 'Borrowings, non current']
columns= pd.MultiIndex.from_product([['2022','2021'], keywords])
data= pd.DataFrame(columns=columns)
    
def extract_info(text,file_name):
    # Input text containing the keyword
    input_text = str(text).split('\\n')
    global keywords
    global data

    # keyword='Total liabilities'
    for keyword in keywords:
    # Use regex to find the keyword and capture text after it
    # pattern = re.compile(f"{re.escape(keyword)}.*?\\b([\d,]+)\\b", re.IGNORECASE)
        pattern = re.compile(f"{re.escape(keyword)}.*?\\b([\d,]+).*?([\d,]+)\\b.*?([\d,]+)\\b", re.IGNORECASE)

        match = pattern.search(str(input_text))
        com_name=file_name.split('_')[1]
        if match:
            if len(match.groups)<=2:
                data.at[com_name,('2022',keyword)]=match.group(1)
                data.at[com_name,('2021',keyword)]=match.group(2)
            elif len(match.group)>2:
                data.at[com_name,('2022',keyword)]=match.group(1)
                data.at[com_name,('2021',keyword)]=match.group(3)
            
    #         captured_number1 = match.group(1)
    #         print(f"Captured number: {captured_number1}")
    #         captured_number2 = match.group(2)
    #         print(f"Capture number: {captured_number2}")
        else:
            print("Keyword not found.")


In [125]:

import pandas as pd

# Specify the folder path
folder_path = 'oman 2022\Balancesheet'

# List of files in the folder
# file_list = os.listdir(folder_path)
# file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
file_list=['2_CMII_BalanceSheet_27022023_2.pdf']
for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)
    print(f"Opening PDF: {file_name}")

    # Open the PDF file
    pdf_file = open(pdf_path, 'rb')

    # Create a PDF reader object
    reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF file
    num_pages = len(reader.pages)

    # Iterate over the pages in the PDF file
    text=[]
    for page_num in range(num_pages):

        # Get the current page
        page = reader.pages[page_num]

        # Get the tables on the current page
        tables = page.extract_text().split('\n\n')

        # Print the tables
        for table in tables:
            text.append(table)
            extract_info(text,file_name)
    # Close the PDF file
    pdf_file.close()
    

Opening PDF: 2_CMII_BalanceSheet_27022023_2.pdf
Keyword not found.
Keyword not found.
Keyword not found.
Keyword not found.
Keyword not found.
Keyword not found.
Keyword not found.
Keyword not found.


In [348]:
def extract_borrow(tables):
    global profit_data
    global file_name
    # Read first table to extract the wanted row
    for i in range(len(tables)):
        
        table=tables[i]
        # Drop null rows from first columns
        first_col=tables[i].columns[0]
        table.dropna(subset=[first_col],inplace=True)
        # Take a loop to look for specific values by order 
        keywords='Borrowings'
    #     columns= ['Cons2022','Stand2022','Cons2021','Stand2021']
    #     profit_data= pd.DataFrame(columns=columns)

    #     for keyword in keywords:
    #     matched_rows=test[test['Unnamed: 0'].str.contains(keyword, case=False)]
        matched_row=table.loc[table[first_col]==keywords]
        com_name=file_name.split('_')[1]

        if not matched_row.empty:
            matched_row=matched_row.iloc[[0]]
            last_columns=matched_row.iloc[:,1:]
            matched_row['combine']=last_columns.apply(lambda row:' '.join(filter(lambda x:pd.notna(x),row)),axis=1)
            matched_row=matched_row['combine'].str.split(expand=True)
            if matched_row.shape[1]==2:
                borrow_data.at[com_name,'Cons2022']=matched_row.iloc[0,0]
                borrow_data.at[com_name,'Cons2021']=matched_row.iloc[0,1]
                break

            elif matched_row.shape[1]>2:
                borrow_data.at[com_name,'Cons2022']=matched_row.iloc[0,0]
                borrow_data.at[com_name,'Cons2021']=matched_row.iloc[0,2]
                break
        else:
            continue

In [347]:
correct_file=[]
wrong_file=[]
# Specify the PDF file path
columns= ['Cons2022','Stand2022','Cons2021','Stand2021']
borrow_data= pd.DataFrame(columns=columns)
folder_path='oman 2022\Balancesheet'
file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
# file_list=['2_AACT_BalanceSheet_26022023_33.pdf']


for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)
#     print(f"Opening PDF: {file_name}")

    # Open the PDF file
    pdf_file = open(pdf_path, 'rb')

# tabula.convert_into("AACT_2022_Yearly (Audited)_e/2_AACT_BalanceSheet_26022023_33.pdf", "output.csv", output_format="csv", pages='all')
# Use tabula to extract tables from the PDF

    tables = read_pdf(pdf_file,multiple_tables=True,columns=[100,500, 500, 500],pages='all')
    # print(tabulate(tables)
    # Print the extracted tables
#     for table_num, table_df in enumerate(tables, start=1):
    result=check_files(tables,file_name)
    if result:
        extract_borrow(tables)
    else:
        continue
#         print(f"Table {table_num}:\n{table_df}")
     # Close the PDF file
    pdf_file.close()

In [353]:
tables[0]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Actuals/Omani Rial/Audited
1,"Statement of Financial position, Current/ non-...",NaN,NaN,NaN
3,STATEMENT OF FINANCIAL POSITION,NaN,NaN,NaN
4,CONSOLIDATED AND SEPARATE,NaN,NaN,NaN
5,ASSETS,NaN,NaN,NaN
6,NON-CURRENT ASSETS,NaN,NaN,NaN
7,"Property, plant and equipment","9,039,348","465,550","9,493,615 524,793"
8,Intangible assets other than goodwill,"504,505",NaN,"412,086"
9,Right-of-use assets,"1,068,288","26,789","1,007,811 32,281"
10,Investments in subsidiaries,NaN,"7,229,520","7,229,520"
11,Deferred tax assets,"1,190,900","527,415","785,166 353,327"


In [198]:
borrow_data.to_excel('borrow2022.xlsx')

In [310]:
tables[1]

,Retained earnings (accumulated Losses),Unnamed: 0,"3,692,636","1,427,085 6,026,832",Unnamed: 1,"3,477,751"
0,Total equity attributable to owners of parent,NaN,"16,600,920","18,370,166",NaN,NaN
1,Non-controlling interests,NaN,"953,239","1,076,050",NaN,NaN
2,Total equity,NaN,"17,554,159","13,638,111 19,446,216",NaN,"15,123,827"
3,LIABILITIES,NaN,NaN,NaN,NaN,NaN
4,NON-CURRENT LIABILITIES,NaN,NaN,NaN,NaN,NaN
5,NON-CURRENT PROVISIONS,NaN,NaN,NaN,NaN,NaN
6,Non-current provisions for employee benefits,NaN,"675,547","120,662 660,799",NaN,"134,572"
7,Total non-current provisions,NaN,"675,547","120,662 660,799",NaN,"134,572"
8,"Borrowings, non current",NaN,NaN,"644,683",NaN,NaN
9,Non-current lease liabilities,NaN,"866,756","19,576 898,810",NaN,"23,516"


In [301]:
first_col=tables[1].columns[0]
tables[1].loc[tables[1][first_col]=='Borrowings, non current']

,CURRENT ASSETS,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
22,"Borrowings, non current","1,868 822",NaN,908.0,908


In [298]:
tables[0].loc[tables[0][tables[0].columns[0]]=='Borrowings, non current'].iloc[:,1:].apply(lambda row:' '.join(filter(lambda x:pd.notna(str(x)),row)),axis=1)

40    1,868 822 908 908
dtype: object

In [226]:
borrow_data[['Cons2022','Cons2021']].columns=[['borrow_non_current(Cons2022)','borrow_non_current(Cons2021)']]

In [231]:
borrow_data.rename(columns={'Cons2022':'borrow_non_current(Cons2022)','Cons2021':'borrow_non_current(Cons2021)'},inplace=True)

In [236]:
borrow_data.iloc[:,[0,2]]

,borrow_non_current(Cons2022),borrow_non_current(Cons2021)
AACT,"3,350,079",0
AFAI,"8,044,333","7,742,493"
CSII,0,0
DBCI,"797,401","638,218"
DFIN,"1,868",908
GECS,"5,968","11,561"
GMPI,"3,741,651","4,424,454"
MGCI,"1,650,000","1,847,500"
MTMI,"21,823","60,223"
NMWI,"697,718","967,193"


In [255]:
borrow_data1.rename(columns={'borrow_non_current(Cons2022)':'borrow_current(Cons2022)','borrow_non_current(Cons2021)':'borrow_current(Cons2021)'},inplace=True)

In [259]:
borrow_final=pd.concat([borrow_data1.iloc[:,[0,2]],borrow_data.iloc[:,[0,2]]],axis=1)

In [323]:
borrow_final.to_excel('oman_borrow_2022.xlsx')

In [322]:
pd.concat([borrow_data,non_borrow],axis=1).head()

,Cons2022,Stand2022,Cons2021,Stand2021,Cons2022,Stand2022,Cons2021,Stand2021
AACT,"773,342",NaN,0,NaN,"3,350,079",NaN,0,NaN
ABRJ,"18,841",NaN,"15,330",NaN,NaN,NaN,NaN,NaN
AFAI,"15,091,690",NaN,"16,061,998",NaN,"8,044,333",NaN,"7,742,493",NaN
AMCI,"600,000",NaN,0,NaN,NaN,NaN,NaN,NaN
ATMI,"25,375,475",NaN,"23,598,796",NaN,NaN,NaN,NaN,NaN


### Finance Cost

In [346]:
def extract_finance(tables):
    global profit_data
    global file_name
    # Read first table to extract the wanted row
    for i in range(len(tables)):
        
        table=tables[i]
        # Drop null rows from first columns
        first_col=tables[i].columns[0]
        table.dropna(subset=[first_col],inplace=True)
        # Take a loop to look for specific values by order 
        keywords='Finance costs'
    #     columns= ['Cons2022','Stand2022','Cons2021','Stand2021']
    #     profit_data= pd.DataFrame(columns=columns)

    #     for keyword in keywords:
    #     matched_rows=test[test['Unnamed: 0'].str.contains(keyword, case=False)]
        matched_row=table.loc[table[first_col]==keywords]
        com_name=file_name.split('_')[1]

        if not matched_row.empty:
            matched_row=matched_row.iloc[[0]]
            last_columns=matched_row.iloc[:,1:]
            matched_row['combine']=last_columns.apply(lambda row:' '.join(filter(lambda x:pd.notna(x),row)),axis=1)
            matched_row=matched_row['combine'].str.split(expand=True)
            if matched_row.shape[1]==2:
                borrow_data.at[com_name,'Cons2022']=matched_row.iloc[0,0]
                borrow_data.at[com_name,'Cons2021']=matched_row.iloc[0,1]
                break

            elif matched_row.shape[1]>2:
                borrow_data.at[com_name,'Cons2022']=matched_row.iloc[0,0]
                borrow_data.at[com_name,'Cons2021']=matched_row.iloc[0,2]
                break
        else:
            continue



In [339]:
correct_file=[]
wrong_file=[]
# Specify the PDF file path
columns= ['Cons2022','Stand2022','Cons2021','Stand2021']
finance_data= pd.DataFrame(columns=columns)
folder_path='oman 2022\balancesheet'
file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
# file_list=['2_AACT_BalanceSheet_26022023_33.pdf']


for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)
#     print(f"Opening PDF: {file_name}")

    # Open the PDF file
    pdf_file = open(pdf_path, 'rb')

# tabula.convert_into("AACT_2022_Yearly (Audited)_e/2_AACT_BalanceSheet_26022023_33.pdf", "output.csv", output_format="csv", pages='all')
# Use tabula to extract tables from the PDF

    tables = read_pdf(pdf_file,multiple_tables=True,columns=[100,500, 500, 500],pages='all')
    # print(tabulate(tables)
    # Print the extracted tables
#     for table_num, table_df in enumerate(tables, start=1):
    result=check_files(tables,file_name)
    if result:
        extract_finance(tables)
    else:
        continue
#         print(f"Table {table_num}:\n{table_df}")
     # Close the PDF file
    pdf_file.close()

In [341]:
wrong_file

['3_NBII_IncomeStatement_06062023_0.pdf',
 '3_OQIC_IncomeStatement_03032023_24.pdf',
 '3_OUIC_IncomeStatement_28022023_14.pdf']

In [333]:
borrow_data.rename(columns={'borrow_current(Cons2022)':'finance_Cons2022','borrow_current(Cons2021)':'finance_Cons2021'},inplace=True)

In [337]:
borrow_data.iloc[:,[0,2]].to_excel('oman_finance_2022.xlsx')

In [340]:
borrow_data.shape

(77, 6)